### For loops to pull L2A, L2B, L4A as needed

In [1]:
!pip install geopandas
!pip install contextily

  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
Using cached contextily-1.6.2-py3-none-any.whl (17 kB)


In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [2]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [4]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [5]:
import geopandas as gpd
import os

def split_gpkg_with_buffer(input_gpkg, output_dir, buffer_distance=0.1):
    # Load the GeoPackage into a GeoDataFrame
    gdf = gpd.read_file(input_gpkg)
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate through each feature (row) in the GeoDataFrame
    for index, row in gdf.iterrows():
        # Extract the 'fid' value (assuming the column name is 'fid')
        fid_value = row['tile_num']
        
        # Create a new GeoDataFrame for the single feature
        single_feature_gdf = gpd.GeoDataFrame([row], crs=gdf.crs)
        single_feature_gdf = single_feature_gdf.to_crs(epsg=4326)
        
        # Add a 0.1 degree buffer to the geometry of the feature
        single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)
        
        # Construct the output file path using the 'fid' value
        output_file = os.path.join(output_dir, f"{fid_value}.gpkg")
        
        # Write the single feature with buffer to its own GeoPackage
        single_feature_gdf.to_file(output_file, driver="GPKG")
        print(f"Saved buffered feature with fid {fid_value} to {output_file}")

# Example usage:
input_gpkg = '/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/Selenge_Boreal_Tiles.gpkg'  # Replace with the path to your input GeoPackage
output_dir = '/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES'  # Replace with the directory where you want to save the split files
split_gpkg_with_buffer(input_gpkg, output_dir)

input_gpkg = '/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/Uvs_Boreal_Tiles.gpkg'  # Replace with the path to your input GeoPackage
output_dir = '/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES'  # Replace with the directory where you want to save the split files
split_gpkg_with_buffer(input_gpkg, output_dir)

/tmp/ipykernel_172/3874193231.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)


Saved buffered feature with fid 2866 to /projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/2866.gpkg


/tmp/ipykernel_172/3874193231.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)


Saved buffered feature with fid 2952 to /projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/2952.gpkg


/tmp/ipykernel_172/3874193231.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)


...

Saved buffered feature with fid 2022 to /projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/2022.gpkg


/tmp/ipykernel_172/3874193231.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)


Saved buffered feature with fid 1953 to /projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1953.gpkg


/tmp/ipykernel_172/3874193231.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  single_feature_gdf['geometry'] = single_feature_gdf['geometry'].buffer(buffer_distance)


KeyboardInterrupt: 

In [5]:
# # Define the folder path containing the tiles
# #folder_path = "/projects/my-public-bucket/AOIs/GHANA/"
# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/"

# # Create a list to store the GeoPackage file paths
# json_files = []

# # Use glob to find all .gpkg files in the folder
# #pattern = os.path.join(folder_path, '*.geojson')
# pattern = os.path.join(folder_path, 'Mali_South.gpkg')
# json_files = glob.glob(pattern)

# # Print the list of GeoPackage file paths
# #for file_path in gpkg_files:
#  #   print(file_path)

# #json_files = [x.replace('/projects/my-public-bucket/AOIs/GHANA/',
# #                        'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/GHANA/') for x in json_files]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/WA_15_countries/') for x in json_files]

# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs[0]

In [6]:
# # OPTIONAL # visually verify the AOI
# aoi = AOIs[0]

# try:
#     import geopandas as gpd
#     import contextily as ctx
# except:
#     print(
#         "If you wish to visually verify your AOI, "
#         "you must install the `geopandas` and `contextily` packages."
#     )
# else:
#     aoi_gdf = gpd.read_file(aoi)
#     aoi_epsg4326 = aoi_gdf.to_crs(epsg=4326)
#     ax = aoi_epsg4326.plot(figsize=(10, 5), alpha=0.3, edgecolor="red")
#     ctx.add_basemap(ax, crs=4326)

In [ ]:
# aoi_name = os.path.basename(AOIs[0]).split('.')[0]
# aoi_name

In [6]:
folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES"
json_files = []
pattern = os.path.join(folder_path, '*.gpkg')
json_files = glob.glob(pattern)
json_files = [ x for x in json_files if "L2A" not in x ]
json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/',
                        'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/') for x in json_files]
print(len(json_files))
print(json_files)
AOIs = json_files

# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/"
# json_files = []
# pattern = os.path.join(folder_path, '*.gpkg')
# json_files = glob.glob(pattern)
# json_files = [ x for x in json_files if "L2A" not in x ]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/') for x in json_files]
# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs

29
['https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1883.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1884.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1952.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1953.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1954.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/1955.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/2021.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/2022.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID_TILES/

In [10]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L2A","L4A"] # ,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
out_dir = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"

AOIs = json_files
#AOIs = [AOIs[2]]

for each_aoi in AOIs: 
    if "L4A" in PRODUCTS: 
        if not os.path.exists(out_dir + f"{os.path.basename(each_aoi).split('.')[0]}_MON_L4A.gpkg"):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L4A",
               lat="lat_lowestmode",
               lon="lon_lowestmode",
               beams="all",
               columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
               query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
               limit = 100000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_MON_L4A.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-32vcpu-64gb",
                username="nehajo88",
                **inputs
            )
            inputs
            job_id = result.id
            job_id or result

    if "L2B" in PRODUCTS: 
        if not os.path.exists(out_dir + f"{os.path.basename(each_aoi).split('.')[0]}_MON_L2B.gpkg"):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L2B",
               lat="geolocation/lat_lowestmode",
               lon="geolocation/lon_lowestmode",
               beams="all",
               columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
               query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
               limit = 100000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_MON_L2B.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-32vcpu-64gb",
                username="nehajo88",
                **inputs
            )
            inputs
            job_id = result.id
            job_id or result
        
        
    if "L2A" in PRODUCTS: 
        if not os.path.exists(out_dir + f"{os.path.basename(each_aoi).split('.')[0]}_MON_L2A.gpkg"):
            aoi_name = os.path.basename(each_aoi).split('.')[0]
            print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
            #aoi_url = each_aoi
            inputs = dict(
               aoi=each_aoi,
               doi="L2A",
               lat="lat_lowestmode",
               lon="lon_lowestmode",
               beams="all",
               columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,sensitivity",
               # columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_f",
               #columns="all",
               query="quality_flag == 1 and sensitivity > 0.95",
               limit = 100000,
               #temporal="-",
               output=f"{os.path.basename(each_aoi).split('.')[0]}_MON_L2A.gpkg"
            )
            result = maap.submitJob(
                identifier="gedi-subset",
                algo_id="gedi-subset",
                version="0.9.0",
                queue="maap-dps-worker-32gb",
                username="nehajo88",
                **inputs
            )
            inputs
            job_id = result.id
            job_id or result
        

Thanks, I'll run the GEDI L2A subsetter for 2093!!


In [11]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/12/04"  # Can now set name
out_dir = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("_MON_L2A.gpkg") or fname.endswith("_MON_L4A.gpkg"):
#        if fname.endswith("NPAs_L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            print(source_path)
            target_path = os.path.join(out_dir, fname)
            print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Moving {fname}...")
                subprocess.call(['mv', source_path, target_path])
            else:
                print(f"{fname} already exists in the target directory. Skipping...")

/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/12/04/20/30/30/564761/2093_MON_L2A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2093_MON_L2A.gpkg
Moving 2093_MON_L2A.gpkg...


In [19]:
import rasterio
import os
import argparse
from rasterio.io import MemoryFile
from rasterio.rio import options
from rasterio.shutil import copy, delete
from rasterio.vrt import WarpedVRT
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.transform import from_origin
from rasterio.crs import CRS
import pandas as pd
import geopandas as gpd
import numpy as np

FOLDER = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"

for dirName, subdirList, fileList in os.walk(FOLDER):
    for fname in fileList:
        if fname.endswith("_MON_L2A.gpkg") and not fname.__contains__("ADM1"):
            OUTFILE = os.path.join(FOLDER, fname.split('.')[0] + '.tif')
            print(OUTFILE)
            
            if not os.path.exists(OUTFILE): 
                L2A = gpd.read_file(os.path.join(FOLDER, fname))
                L2A = L2A.drop('geometry', axis=1)
                
                x = L2A["lon_lowestmode"]
                y = L2A["lat_lowestmode"]
                z = L2A.rh98
                # z[z == 0] = np.nan  # Replace 0 values with NaN for 'no data' handling
                
                pixel_width = 0.004  # units in degrees
                pixel_height = 0.004  # units in degrees
                
                nx = int(np.round((np.max(x) - np.min(x)) / pixel_width))  # x size of output raster
                ny = int(np.round((np.max(y) - np.min(y)) / pixel_width))  # y size of output raster
                
                zi, yi, xi = np.histogram2d(y, x, bins=(ny, nx), weights=z)
                counts, _, _ = np.histogram2d(y, x, bins=(ny, nx))
                
                # Calculate mean value (avoid divide by 0 errors)
                zi = np.divide(zi, counts, out=np.zeros_like(zi), where=counts != 0)
                
                # Reverse array to match coordinate system
                z = np.array(zi.data[::-1])
                
                # Set NaN values to 0 in the array (nodata handling)
                z[np.isnan(z)] = 0  # Replace NaN values with 0

                pixel_width = xi[1] - xi[0]  # Approximate pixel width
                pixel_height = yi[1] - yi[0]  # Approximate pixel height
                x_min = xi[0]
                y_max = yi[-1]
                
                transform = from_origin(x_min, y_max, pixel_width, pixel_height)
                crs = CRS.from_epsg(4326)
                
                kwargs = {
                    'driver': 'GTiff',
                    'dtype': z.dtype,
                    'count': 1,  # Number of bands
                    'height': z.shape[0],
                    'width': z.shape[1],
                    'crs': crs,
                    'transform': transform,
                    'predictor': 2,
                    'nodata': 0  # Set the nodata value to 0
                }
                
                with MemoryFile() as memfile:
                    with memfile.open(**kwargs) as mem:
                        # Write data with 'nodata' set to 0
                        mem.write(z.reshape(1, z.shape[0], z.shape[1]))
                
                        dst_profile = cog_profiles.get("deflate")
                        cog_translate(
                            mem,
                            OUTFILE,
                            dst_profile,
                            in_memory=False
                        )
                
                print("GeoTIFF has been created successfully with GDAL.")
            else:
                print("GeoTIFF exists")

/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1883_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1884_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1952_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1953_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1954_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/1955_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2021_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2022_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2023_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2024_MON_L2A.tif
GeoTIFF exists
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/2091_MON_L2A.tif
Geo

In [44]:
import subprocess
import os
import numpy as np
from osgeo import gdal

def merge_rasters_with_average(input_files, output_file, data_type='Float32', compression='DEFLATE', predictor=2, zlevel=9):
    # Create a temporary directory for the optfile
    temp_dir = os.path.dirname(output_file)
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    
    # Create the optfile containing the list of input files
    optfile_path = os.path.join(temp_dir, 'mergeInputFiles.txt')
    with open(optfile_path, 'w') as optfile:
        for file in input_files:
            optfile.write(f"{file}\n")
    
    # Prepare the gdal_merge.py command to merge the rasters
    gdal_merge_command = [
        'gdal_merge.py',
        '-ot', data_type,
        '-of', 'GTiff',
        '-co', f'COMPRESS={compression}',
        '-co', f'PREDICTOR={predictor}',
        '-co', f'ZLEVEL={zlevel}',
        '-o', output_file,
        '--optfile', optfile_path
    ]
    
    # Run the command using subprocess
    try:
        subprocess.run(gdal_merge_command, check=True)
        print(f"Merge successful. Output saved to: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during merge: {e}")
    finally:
        # Clean up the optfile
        if os.path.exists(optfile_path):
            os.remove(optfile_path)
    
    # Now, we average the overlapping regions, ignoring NoData values (treated as 0)
    average_raster(output_file, input_files)


def average_raster(merged_raster_path, input_files):
    # Open the merged raster using GDAL
    ds = gdal.Open(merged_raster_path, gdal.GA_Update)
    if ds is None:
        print(f"Failed to open raster: {merged_raster_path}")
        return

    # Read raster data into an array
    band = ds.GetRasterBand(1)  # Assuming a single band raster
    raster_data = band.ReadAsArray()

    # Get the raster dimensions
    rows, cols = raster_data.shape

    # Create an empty array for the averaged values and count of valid pixels
    average_data = np.zeros_like(raster_data, dtype=np.float32)
    count_data = np.zeros_like(raster_data, dtype=np.int32)

    # Loop through the input rasters to get their NoData values
    nodata_values = []
    for file in input_files:
        src_ds = gdal.Open(file)
        if src_ds is None:
            print(f"Failed to open raster: {file}")
            continue
        band = src_ds.GetRasterBand(1)  # Assuming single band raster
        nodata = band.GetNoDataValue()
        nodata_values.append(nodata)

    # Iterate over the merged raster data to perform averaging
    for i in range(rows):
        for j in range(cols):
            values_to_average = []
            for idx, file in enumerate(input_files):
                src_ds = gdal.Open(file)
                band = src_ds.GetRasterBand(1)
                data = band.ReadAsArray()
                nodata = nodata_values[idx]

                value = data[i, j]

                # If the value is NoData (or 0, since we convert NoData to 0), skip it
                if nodata is not None and value == nodata:  # Ignore NoData
                    value = 0  # Convert NoData to 0
                if value != 0:  # Ignore 0 values (representing NoData in this context)
                    values_to_average.append(value)

            # If we have valid values, compute the average
            if values_to_average:
                average_data[i, j] = np.mean(values_to_average)
                count_data[i, j] = len(values_to_average)

    # Set the NoData value for pixels where there were no valid data points
    average_data[count_data == 0] = -9999  # Adjust NoData value if needed

    # Write the averaged data back to the raster
    band.WriteArray(average_data)
    band.SetNoDataValue(-9999)  # Set the NoData value
    ds = None
    print(f"Averaging complete. The output is saved in {merged_raster_path}")

# if __name__ == "__main__":
#     folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"
#     json_files = []
#     pattern = os.path.join(folder_path, '*MON_L2A.tif')
#     input_files = glob.glob(pattern)[0:16]
#     input_files
#     output_file = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/GEDI_L2A_Selenge.tif"
    
#     if not os.path.exists(output_file): merge_rasters_with_average(input_files, output_file)
    
if __name__ == "__main__":
    folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"
    json_files = []
    pattern = os.path.join(folder_path, '*MON_L2A.tif')
    input_files = glob.glob(pattern)[16:]
    input_files
    output_file = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/GEDI_L2A_Uvs.tif"
    
    if not os.path.exists(output_file): merge_rasters_with_average(input_files, output_file)
    

0...10...20...30...40...50...60...70...80...90...100 - done.
Merge successful. Output saved to: /projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/GEDI_L2A_Uvs.tif


IndexError: index 58 is out of bounds for axis 1 with size 58